<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/homeexercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding, AutoTokenizer

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }



# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [ ]:
!pip install optuna

  Using cached optuna-4.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.6-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.1.0-py3-none-any.whl (364 kB)
Using cached alembic-1.14.0-py3-none-any.whl (233 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached Mako-1.3.6-py3-none-any.whl (78 kB)


In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer", # where to put everything that is trained
    per_device_train_batch_size=8,  # amount of sentences model looks at before anything is updated
    per_device_eval_batch_size=8,
    logging_steps=8,                # how many times results are logged/track/output
    num_train_epochs=10,
    eval_strategy="epoch",          # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=1e-5,             # influences how weight changes in function, determines how big a step you take in valley
    weight_decay=0.005,              # sum of weights added to loss function, regulization method, penalizes large weights
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.682200,0.693174,0.468750
2,0.684100,0.688901,0.468750
3,0.663700,0.672419,0.500000
4,0.612400,0.644293,0.781250
5,0.562400,0.607863,0.812500
6,0.462400,0.553763,0.843750
7,0.375600,0.513987,0.812500
8,0.314200,0.478871,0.875000
9,0.267300,0.463749,0.875000
10,0.272500,0.459768,0.843750


TrainOutput(global_step=160, training_loss=0.5006450787186623, metrics={'train_runtime': 1239.8815, 'train_samples_per_second': 1.032, 'train_steps_per_second': 0.129, 'total_flos': 36337463293824.0, 'train_loss': 0.5006450787186623, 'epoch': 10.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[-0.06958379,  0.06051268],
       [-0.08787943,  0.04495766],
       [-0.06524669,  0.03900277],
       [-0.09573183,  0.04293787],
       [-0.09416903,  0.05197893],
       [-0.07838155,  0.05938193],
       [-0.10323695,  0.06147411],
       [-0.07801252,  0.03898965],
       [-0.09623865,  0.05653071],
       [-0.08586179,  0.05698165],
       [-0.10648751,  0.06719922],
       [-0.0948368 ,  0.07416285],
       [-0.11036998,  0.06861213],
       [-0.0839076 ,  0.03644176],
       [-0.10452615,  0.02931933],
       [-0.09425898,  0.06834401],
       [-0.08783385,  0.05719883],
       [-0.05314425,  0.02714493],
       [-0.07604894,  0.0246217 ],
       [-0.08113252,  0.05615607],
       [-0.10356317,  0.07720418],
       [-0.09844141,  0.08528958],
       [-0.08160684,  0.0438307 ],
       [-0.09800542,  0.05818297],
       [-0.09593192,  0.03866607],
       [-0.10241057,  0.03744058],
       [-0.06861857,  0.0259532 ],
       [-0.09238237,  0.01

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [ ]:
import torch

test_str = "I hate this movie!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/checkpoint-144")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

NEGATIVE


In [ ]:
test_dataset = small_imdb_dataset['test']

test_results = trainer.predict(small_tokenized_dataset['test'])
print("Test set evaluation results:", test_results)

Test set evaluation results: PredictionOutput(predictions=array([[-0.09830225,  0.05275302],
       [-0.06768271,  0.05623597],
       [-0.08936425,  0.04213354],
       [-0.09573521,  0.07227322],
       [-0.05934578,  0.04540213],
       [-0.09757014,  0.04447775],
       [-0.11173826,  0.05679902],
       [-0.09931773,  0.03220566],
       [-0.09505644,  0.03322481],
       [-0.09729728,  0.05454333],
       [-0.06769577,  0.03111251],
       [-0.0958791 ,  0.05999861],
       [-0.08119949,  0.02702319],
       [-0.11605772,  0.05850192],
       [-0.0888561 ,  0.0325653 ],
       [-0.07517722,  0.05054054],
       [-0.08820112,  0.02393899],
       [-0.08104749,  0.07618146],
       [-0.08095472,  0.05318353],
       [-0.09145476,  0.0463153 ],
       [-0.09330715,  0.04325824],
       [-0.07009041,  0.00871004],
       [-0.07560414,  0.049671  ],
       [-0.06530508,  0.03899768],
       [-0.09336028,  0.04417555],
       [-0.08097111,  0.03776117],
       [-0.07498656,  0.04667438

In [ ]:
tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)

# new trainer for evaluation
trainer = Trainer(
    model=fine_tuned_model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['test'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print(test_results)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

{'eval_loss': 0.5175219774246216, 'eval_model_preparation_time': 0.0033, 'eval_accuracy': 0.84375, 'eval_runtime': 12.3077, 'eval_samples_per_second': 2.6, 'eval_steps_per_second': 0.325}


In [ ]:
test_results = trainer.evaluate(small_tokenized_dataset['test'])
print("Test set evaluation results:", test_results)

Test set evaluation results: {'eval_loss': 0.5175219774246216, 'eval_model_preparation_time': 0.0033, 'eval_accuracy': 0.84375, 'eval_runtime': 7.593, 'eval_samples_per_second': 4.214, 'eval_steps_per_second': 0.527}
